## import library

In [57]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# using machine learning tensorflow
import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.5.0


## Collection Data

In [58]:
gender = pd.read_csv("/kaggle/input/titanic/gender_submission.csv",delimiter=',')
test = pd.read_csv("/kaggle/input/titanic/test.csv",delimiter=',')
train = pd.read_csv("/kaggle/input/titanic/train.csv",delimiter=',')

In [59]:
gender.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [60]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [61]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Understanding

In [62]:
print(train.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


Fitur manakah yang termasuk data kategorial?

Nilai-nilai ini mengklasifikasikan sampel ke dalam kumpulan sampel serupa. Dalam fitur kategoris apakah nilai didasarkan pada nominal, ordinal, rasio, atau interval? Hal ini antara lain membantu kami memilih plot yang sesuai untuk visualisasi.

Kategorikal: Survived, Sex, and Embarked. Ordinal: Pclass.

Fitur manakah yang termauk data numerik?

Fitur manakah yang bersifat numerik? Nilai-nilai ini berubah dari sampel ke sampel. Dalam fitur numerik, apakah nilainya bersifat diskrit, kontinu, atau berdasarkan deret waktu? Hal ini antara lain membantu kami memilih plot yang sesuai untuk visualisasi.

Data kontinu & diskrit: Age, Fare. Discrete: SibSp, Parch.

In [63]:
# preview the data
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Fitur yang memiliki tipe data campuran adalah fitur yang mengandung kombinasi data numerik dan data alfanumerik (teks) dalam satu fitur. Ini adalah kandidat yang perlu diperbaiki.

"Ticket" adalah campuran dari tipe data numerik dan alfanumerik. "Cabin" adalah alfanumerik/character

Fitur-fitur yang mungkin mengandung kesalahan atau typo adalah fitur-fitur di mana data tidak konsisten atau mungkin dimasukkan secara tidak akurat. Ini bisa sulit untuk ditinjau dalam dataset besar, tetapi dengan meninjau beberapa sampel dari dataset yang lebih kecil, kita mungkin bisa mengetahui dengan jelas fitur-fitur mana yang memerlukan koreksi.

Fitur "Name" mungkin mengandung kesalahan atau typo karena ada beberapa cara yang digunakan untuk mendeskripsikan nama, termasuk gelar, tanda kurung bulat, dan tanda kutip yang digunakan untuk nama alternatif atau singkat. Fitur ini bisa rentan terhadap inkonsistensi dan variasi dalam penginputan data, sehingga penting untuk meninjau dan potensialnya memperbaikinya.

In [64]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


Fitur-fitur yang mengandung nilai kosong, null, atau kosong adalah fitur-fitur yang perlu diperbaiki.

Pada dataset pelatihan, fitur-fitur "Cabin" > "Age" > "Embarked" mengandung sejumlah nilai null, dalam urutan tersebut.
Pada dataset uji, fitur "Cabin" > "Age" tidak lengkap.

Berikut adalah tipe data untuk berbagai fitur:

Membantu kita selama proses konversi.

Tujuh fitur adalah integer atau float. Enam fitur dalam kasus dataset uji.
Lima fitur adalah string (objek).

# Exploring Data And Glimpsy Data

In [65]:
#info data

In [66]:
train.info()
print("----")
test.info()
print("----")
gender.info()
print("----")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

# Duplicated Data

In [67]:
train.duplicated().sum()

0

In [68]:
test.duplicated().sum()

0

# Missing Value

In [69]:
train.isnull().value_counts()

PassengerId  Survived  Pclass  Name   Sex    Age    SibSp  Parch  Ticket  Fare   Cabin  Embarked
False        False     False   False  False  False  False  False  False   False  True   False       529
                                                                                 False  False       183
                                             True   False  False  False   False  True   False       158
                                                                                 False  False        19
                                             False  False  False  False   False  False  True          2
Name: count, dtype: int64

In [70]:
test.isnull().value_counts()

PassengerId  Pclass  Name   Sex    Age    SibSp  Parch  Ticket  Fare   Cabin  Embarked
False        False   False  False  False  False  False  False   False  True   False       244
                                                                       False  False        87
                                   True   False  False  False   False  True   False        82
                                                                       False  False         4
                                   False  False  False  False   True   True   False         1
Name: count, dtype: int64

In [71]:
gender.isnull().value_counts()

PassengerId  Survived
False        False       418
Name: count, dtype: int64

# Unvariate Analyst

## Stastistik Despritive

In [72]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Nama (Name): Nama-nama unik di seluruh dataset (jumlah=unik=891).

Jenis Kelamin (Sex): Variabel jenis kelamin memiliki dua nilai mungkin, dengan 65% pria (terbanyak=laki-laki, frekuensi=577/jumlah=891).

Kabin (Cabin): Nilai-nilai Kabin memiliki beberapa duplikasi di seluruh sampel. Alternatifnya, beberapa penumpang berbagi kabin.

Tempat Berangkat (Embarked): Tempat berangkat memiliki tiga nilai mungkin. Tempat S digunakan oleh sebagian besar penumpang (terbanyak=S).

Tiket (Ticket): Fitur Tiket memiliki rasio duplikasi yang tinggi (22%), dengan nilai unik sebanyak 681.

In [73]:
train.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


In [74]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [75]:
#gender.describe()

# Biverate Analyst

# Menganalisis dengan memutar fitur-fitur

Untuk mengonfirmasi beberapa observasi dan asumsi kita, kita dapat dengan cepat menganalisis korelasi fitur dengan memutar fitur satu sama lain. Kita hanya bisa melakukannya pada tahap ini untuk fitur-fitur yang tidak memiliki nilai yang kosong. Juga, hanya masuk akal untuk melakukan ini pada fitur-fitur yang bersifat kategori (Sex), ordinal (Pclass), atau diskrit (SibSp, Parch).

Pclass
Kita mengamati korelasi signifikan (>0.5) antara Pclass=1 dan Survived (mengklasifikasikan #3). Kita memutuskan untuk menyertakan fitur ini dalam model kita.

Sex
Kita mengkonfirmasi observasi selama perumusan masalah bahwa Sex=perempuan memiliki tingkat kelangsungan hidup yang sangat tinggi, yaitu 74% (mengklasifikasikan #1).

SibSp dan Parch
Fitur-fitur ini memiliki korelasi nol untuk beberapa nilai tertentu. Mungkin yang terbaik adalah mendapatkan fitur atau serangkaian fitur dari fitur-fitur individu ini (menciptakan #1).

In [76]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [77]:
train[["SibSp","Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived',ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [78]:
train[["SibSp","Survived"]].groupby(['SibSp'],as_index=False).mean().sort_values(by='Survived',ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [79]:
train[['Parch','Survived']].groupby(['Parch'],as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


# Bivarate Analyze by visualizing data


## Korelasi antar Fitur Numerik
Grafik histogram berguna untuk menganalisis variabel numerik kontinu seperti Usia di mana pengelompokan atau rentang akan membantu mengidentifikasi pola yang berguna. Histogram dapat mengindikasikan distribusi sampel menggunakan interval yang didefinisikan secara otomatis atau interval yang sama. Ini membantu kita menjawab pertanyaan yang berkaitan dengan interval tertentu (Apakah bayi memiliki tingkat kelangsungan hidup yang lebih tinggi?)

Perhatikan bahwa sumbu x dalam visualisasi histogram mewakili jumlah sampel atau penumpang.

Observasi.

Bayi (Usia <=4) memiliki tingkat kelangsungan hidup yang tinggi.
Penumpang tertua (Usia = 80) selamat.
Sejumlah besar orang berusia 15-25 tahun tidak selamat.
Sebagian besar penumpang berada dalam rentang usia 15-35 tahun.
Keputusan.

Analisis sederhana ini mengkonfirmasi asumsi kita sebagai dasar keputusan untuk tahap selanjutnya dalam alur kerja.

Kita harus mempertimbangkan Usia (asumsi kita dalam mengklasifikasikan #2) dalam pelatihan model kita.
Lengkapkan fitur Usia untuk nilai-nilai yang hilang (melengkapi #1).
Kita harus mengelompokkan kelompok usia (menciptakan #3).

In [80]:
fig = px.histogram(train, x='Age', color='Survived',
                   barmode='overlay',
                   nbins=20,
                   title='Distribusi Usia berdasarkan Status Selamat',
                   labels={'Age': 'Usia', 'Survived': 'Selamat'})

fig.show()

## Mengkorelasikan fitur numerik dan ordinal

can combine multiple features for identifying correlations using a single plot. This can be done with numerical and categorical features which have numeric values.

Observations.

Pclass=3 had most passengers, however most did not survive. Confirms our classifying assumption #2.
Infant passengers in Pclass=2 and Pclass=3 mostly survived. Further qualifies our classifying assumption #2.
Most passengers in Pclass=1 survived. Confirms our classifying assumption #3.
Pclass varies in terms of Age distribution of passengers.
Decisions.

Consider Pclass for model training.

In [81]:
# Menyiapkan subplot. Ada tiga kelas ('Pclass') dan dua kategori dalam 'Survived'
rows = len(train['Pclass'].unique())
cols = len(train['Survived'].unique())

fig = make_subplots(rows=rows, cols=cols, subplot_titles=('Survived: 0', 'Survived: 1'))

# Mengisi subplot dengan histogram
for i, pclass in enumerate(train['Pclass'].unique()):
    for j, survived in enumerate(train['Survived'].unique()):
        filtered_df = train[(train['Pclass'] == pclass) & (train['Survived'] == survived)]
        fig.add_trace(
            go.Histogram(x=filtered_df['Age'], nbinsx=20, name=f'Pclass {pclass} - Survived {survived}', opacity=0.5),
            row=i+1, col=j+1
        )

# Menambahkan judul dan legenda
fig.update_layout(height=600, width=800, title_text="Distribusi Usia berdasarkan Kelas dan Status Selamat")
fig.update_traces(marker=dict(line=dict(width=0.5, color='black')))
fig.show()

# Korelasi fitur data kategorial

Pengamatan.

Penumpang perempuan memiliki tingkat kelangsungan hidup yang jauh lebih baik dibandingkan laki-laki. Mengonfirmasi klasifikasi (#1).
Pengecualian di Embarked=C di mana laki-laki memiliki tingkat kelangsungan hidup yang lebih tinggi. Ini bisa jadi merupakan korelasi antara Pclass dan Embarked dan pada gilirannya Pclass dan Survived, belum tentu merupakan korelasi langsung antara Embarked dan Survived.
Laki-laki memiliki tingkat kelangsungan hidup yang lebih baik di kelas P=3 jika dibandingkan dengan kelas P=2 untuk port C dan Q. Menyelesaikan (#2).
Pelabuhan embarkasi memiliki tingkat kelangsungan hidup yang bervariasi untuk kelas P=3 dan di antara penumpang pria. Berkorelasi (#1).

Keputusan.

Tambahkan fitur Seks ke pelatihan model.
Selesaikan dan tambahkan fitur Embarked ke pelatihan model.

In [82]:
# First, we need to prepare the data for plotting
df = train.groupby(['Embarked', 'Pclass', 'Sex'])['Survived'].mean().reset_index()

# Identify unique 'Embarked' values for creating subplots
embarked_values = df['Embarked'].unique()

# Create subplots
fig = make_subplots(rows=len(embarked_values), cols=1, subplot_titles=embarked_values)

# Add traces
for i, embarked in enumerate(embarked_values, 1):
    df_embarked = df[df['Embarked'] == embarked]
    for sex in df['Sex'].unique():
        df_sex = df_embarked[df_embarked['Sex'] == sex]
        fig.add_trace(go.Scatter(x=df_sex['Pclass'], y=df_sex['Survived'], 
                                 mode='lines+markers', name=sex), 
                      row=i, col=1)

# Update layout
fig.update_layout(height=600, width=800, title_text="Survival Rate by Class and Embarked Port")
fig.show()

## Mengkorelasikan data kategorial dan numerik
Kami mungkin juga ingin mengkorelasikan fitur kategoris (dengan nilai non-numerik) dan fitur numerik. Kita dapat mempertimbangkan untuk mengkorelasikan Embarked (Non-numerik kategoris), Jenis Kelamin (Non-numerik kategoris), Tarif (Numerik kontinu), dengan Bertahan (Numerik kategoris).

Pengamatan.

Penumpang yang membayar tarif lebih tinggi memiliki kelangsungan hidup yang lebih baik. Mengonfirmasi asumsi kami untuk membuat rentang tarif (#4).
Pelabuhan embarkasi berkorelasi dengan tingkat kelangsungan hidup. Konfirmasi korelasi (#1) dan penyelesaian (#2).
Keputusan.

Pertimbangkan fitur banding Tarif.

In [83]:
df = train.groupby(['Embarked', 'Survived', 'Sex'])['Fare'].mean().reset_index()

# Create the facet grid plot
fig = px.bar(df, x='Sex', y='Fare', facet_row='Embarked', facet_col='Survived', 
             color='Sex', barmode='group', 
             category_orders={"Embarked": sorted(df['Embarked'].unique())},
             height=600, width=800)

# Update layout
fig.update_layout(title_text="Average Fare by Sex, Embarked, and Survival Status")
fig.update_traces(marker=dict(line=dict(width=0), opacity=0.5))
fig.show()

# Data Preprocessing

saya akan menerapkan preprocessing dan transformasi berikut pada kumpulan data titanic ini

Tokenisasi nama-namanya. Misalnya, "Braund, Mr. Owen Harris" akan menjadi ["Braund", "Mr.", "Owen", "Harris"].
Ekstrak awalan apa pun di tiket. Misalnya tiket "STON/O2.3101282" akan menjadi "STON/O2." dan 3101282.

In [129]:
def preprocess(df):
    df = train.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
preprocessed_train_df = preprocess(train)
preprocessed_test_df = preprocess(test)

preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [130]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


# Convert dataset to TensorFlow Dataset

In [131]:
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

kode ini adalah bagian dari pipeline prapemrosesan data untuk model pembelajaran mesin menggunakan TensorFlow. Hal ini terutama berfokus pada pemrosesan fitur 'Nama' dengan melakukan tokenisasi, yang dapat bermanfaat untuk model yang struktur atau komponen namanya relevan (seperti dalam beberapa tugas pemrosesan bahasa alami). train_ds dimaksudkan untuk melatih model, dan serve_ds kemungkinan dimaksudkan untuk membuat prediksi dengan model yang dilatih.

# Train model using default parameters

In [132]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[WARNING 23-12-15 05:25:05.0825 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:25:05.0825 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:25:05.0825 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:25:05.7512 UTC kernel.cc:1243] Loading model from path /tmp/tmpn_8dd7oj/model/ with prefix 18cee2e4b3c14fd5
[INFO 23-12-15 05:25:05.7582 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-12-15 05:25:05.7583 UTC kernel.cc:1075] Use fast generic engine


Accuracy: 0.8260869383811951 Loss:0.8608942627906799


# Train model using improved default parameters

In [133]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples=1,
    categorical_algorithm="RANDOM",
    #max_depth=4,
    shrinkage=0.05,
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[WARNING 23-12-15 05:25:10.2807 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:25:10.2807 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:25:10.2807 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:25:11.2408 UTC kernel.cc:1243] Loading model from path /tmp/tmpaqebq_o6/model/ with prefix abc8fe94f6024f60
[INFO 23-12-15 05:25:11.2514 UTC decision_forest.cc:660] Model loaded with 42 root(s), 2212 node(s), and 10 input feature(s).
[INFO 23-12-15 05:25:11.2515 UTC kernel.cc:1075] Use fast generic engine


Accuracy: 0.782608687877655 Loss:1.060815453529358


# Model SUMMARY

In [134]:
model.summary()

Model: "gradient_boosted_trees_model_113"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.597073 ################
    2.           "Age"  0.363764 #######
    3.          "Fare"  0.264018 ###
    4.          "Name"  0.207843 #
    5.        "Pclass"  0.178906 
    6. "Ticket_number"  0.178488 
    7.   "Ticket_item"  0.177907 
    8.      "Embarked"  0.177237 
    9.         "Parch"  0.175481 
   10.         "SibSp"  0.171800 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 36.000000 ####

# Prediction

In [107]:
pip install tensorflow-decision-forests

Note: you may need to restart the kernel to use updated packages.


In [108]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [140]:
def prediction_to_kaggle_format(model, test_ds, threshold=0.5):
    # Make predictions
    predictions = model.predict(test_ds)

    # In binary classification, TFDF models usually output the probability of the positive class directly
    proba_survive = predictions[:, 0]

    # Convert predictions to binary format based on the threshold
    survived = (proba_survive >= threshold).astype(int)

    return pd.DataFrame({
        "PassengerId": preprocessed_test_df["PassengerId"],
        "Survived": survived
    })

# Generate predictions for submission
kaggle_predictions = prediction_to_kaggle_format(model, test_ds)

1/1 [==============================] - 0s 241ms/step


In [141]:
def make_submission(kaggle_predictions):
    path = "/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

# Generate the submission file
make_submission(kaggle_predictions)

Submission exported to /kaggle/working/submission.csv


# Train Model Using Hyperparameter Tuning

In [142]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

Use /tmp/tmpqtn2ivkl as temporary training directory


[WARNING 23-12-15 05:31:37.2715 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:31:37.2716 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:31:37.2716 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:33:21.4787 UTC kernel.cc:1243] Loading model from path /tmp/tmpqtn2ivkl/model/ with prefix a6a1ecfc72af4358
[INFO 23-12-15 05:33:21.4901 UTC decision_forest.cc:660] Model loaded with 32 root(s), 852 node(s), and 12 input feature(s).
[INFO 23-12-15 05:33:21.4901 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-12-15 05:33:21.4902 UTC kernel.cc:1075] Use fast generic engine


Accuracy: 0.8630136847496033 Loss:0.6804219484329224


Model Mengalami peninggkatan 
Accuracy: 0.8630136847496033 Loss:0.6804219484329224
add Codeadd Markdown

# Make ML model with Ensemble

Di sini saya akan membuat 100 model dengan benih berbeda dan menggabungkan hasilnya

Pendekatan ini menghilangkan sedikit aspek acak yang terkait dengan pembuatan model ML

Dalam pembuatan GBT digunakan parameter jujur. Ini akan menggunakan contoh pelatihan yang berbeda untuk menyimpulkan struktur dan nilai daun. Teknik regularisasi ini menukar contoh dengan perkiraan bias.

In [143]:
predictions = None
num_predictions = 0

for i in range(100):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, # Very few logs
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True, # Only use the features in "features"

        #min_examples=1,
        #categorical_algorithm="RANDOM",
        ##max_depth=4,
        #shrinkage=0.05,
        ##num_candidate_attributes_ratio=0.2,
        #split_axis="SPARSE_OBLIQUE",
        #sparse_oblique_normalization="MIN_MAX",
        #sparse_oblique_num_projections_exponent=2.0,
        #num_trees=2000,
        ##validation_ratio=0.0,
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(test_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": preprocessed_test_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)

i:0


[WARNING 23-12-15 05:33:21.8338 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:33:21.8338 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:33:21.8339 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:33:22.3556 UTC kernel.cc:1243] Loading model from path /tmp/tmp6hduypey/model/ with prefix fa021c7a0d0b40f3
[INFO 23-12-15 05:33:22.3594 UTC kernel.cc:1075] Use fast generic engine


i:1


[WARNING 23-12-15 05:33:22.8788 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:33:22.8788 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:33:22.8789 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:33:23.7119 UTC kernel.cc:1243] Loading model from path /tmp/tmp1128qosk/model/ with prefix 75a05556cf5b4d30
[INFO 23-12-15 05:33:23.7294 UTC kernel.cc:1075] Use fast generic engine


i:2


...

i:98


[WARNING 23-12-15 05:35:25.2839 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:35:25.2839 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:35:25.2839 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:35:25.9103 UTC kernel.cc:1243] Loading model from path /tmp/tmp3wlolqtf/model/ with prefix 6cb058577b7b42ff
[INFO 23-12-15 05:35:25.9171 UTC kernel.cc:1075] Use fast generic engine


i:99


[WARNING 23-12-15 05:35:26.4414 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:35:26.4414 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-15 05:35:26.4414 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-15 05:35:27.2280 UTC kernel.cc:1243] Loading model from path /tmp/tmp5_b2chag/model/ with prefix b6dd2775a47a49ad
[INFO 23-12-15 05:35:27.2410 UTC kernel.cc:1075] Use fast generic engine


Submission exported to /kaggle/working/submission.csv
